# Lesson 1: Simple ReAct Agent from Scratch

In [1]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url='http://127.0.0.1:11434/v1',
    # sk-xxx替换为自己的key
    api_key='sk-xxx'
)

In [3]:
chat_completion = client.chat.completions.create(
    #model="deepseek-r1:32b",
    model="qwen2",
    messages=[{"role": "user", "content": "你好"}]
)

In [4]:
chat_completion.choices[0].message.content

'你好！很高兴能为你服务。有什么问题需要我帮忙解答吗？'

In [5]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        #model="deepseek-r1:32b",
                        model="qwen2",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [6]:
prompt = """
你在一个 Thought（思考）、Action（行动）、PAUSE（暂停）、Observation（观察）的循环中运行。
在循环结束时，你输出一个 Answer（答案）。
使用 Thought 来描述你对你被问到的问题的想法。
使用 Action 来执行你可用的操作之一 - 然后返回 PAUSE。
Observation 将是你执行这些操作的结果。

你可用的操作有：

calculate（计算）：
例如：calculate: 4 * 7 / 3
运行一个计算并返回结果 - 使用 Python，所以如果需要的话请确保使用浮点数语法

average_dog_weight（犬种平均体重）：
例如：average_dog_weight: 牧羊犬
当给出犬种时，返回狗的平均体重

示例对话：

问题：斗牛犬的体重是多少？
思考：我应该用 average_dog_weight 查找狗的体重
行动：average_dog_weight: 斗牛犬
PAUSE（暂停）

你将会再次被这样调用：

观察：斗牛犬的体重是51磅

然后你输出：

答案：斗牛犬的体重是51磅
""".strip()

In [7]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "苏格兰梗犬": 
        return("苏格兰梗犬的平均体重是20磅。")
    elif name in "边境牧羊犬":
        return("边境牧羊犬的平均体重是37磅")
    elif name in "玩具贵宾犬":
        return("玩具贵宾犬的平均体重是7磅")
    else:
        return("狗的平均体重是50磅")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [8]:
abot = Agent(prompt)

In [9]:
result = abot("玩具贵宾犬的体重是多少？")
print(result)

思考：我应该用 average_dog_weight 查找狗的体重。
行动：average_dog_weight: 玩具贵宾犬
PAUSE（暂停）


In [10]:
result = average_dog_weight("玩具贵宾犬")

In [11]:
result

'玩具贵宾犬的平均体重是7磅'

In [12]:
next_prompt = "观察: {}".format(result)

In [13]:
abot(next_prompt)

'答案：玩具贵宾犬的体重是7磅。'

In [14]:
abot.messages

[{'role': 'system',
  'content': '你在一个 Thought（思考）、Action（行动）、PAUSE（暂停）、Observation（观察）的循环中运行。\n在循环结束时，你输出一个 Answer（答案）。\n使用 Thought 来描述你对你被问到的问题的想法。\n使用 Action 来执行你可用的操作之一 - 然后返回 PAUSE。\nObservation 将是你执行这些操作的结果。\n\n你可用的操作有：\n\ncalculate（计算）：\n例如：calculate: 4 * 7 / 3\n运行一个计算并返回结果 - 使用 Python，所以如果需要的话请确保使用浮点数语法\n\naverage_dog_weight（犬种平均体重）：\n例如：average_dog_weight: 牧羊犬\n当给出犬种时，返回狗的平均体重\n\n示例对话：\n\n问题：斗牛犬的体重是多少？\n思考：我应该用 average_dog_weight 查找狗的体重\n行动：average_dog_weight: 斗牛犬\nPAUSE（暂停）\n\n你将会再次被这样调用：\n\n观察：斗牛犬的体重是51磅\n\n然后你输出：\n\n答案：斗牛犬的体重是51磅'},
 {'role': 'user', 'content': '玩具贵宾犬的体重是多少？'},
 {'role': 'assistant',
  'content': '思考：我应该用 average_dog_weight 查找狗的体重。\n行动：average_dog_weight: 玩具贵宾犬\nPAUSE（暂停）'},
 {'role': 'user', 'content': '观察: 玩具贵宾犬的平均体重是7磅'},
 {'role': 'assistant', 'content': '答案：玩具贵宾犬的体重是7磅。'}]

In [15]:
abot = Agent(prompt)

In [16]:
question = """我有两只狗，一只边境牧羊犬和一只苏格兰梗犬。它们的总重量是多少？"""
abot(question)

'思考：我需要计算两只狗的总重量。\n行动：calculate: 2 * average_dog_weight: 边境牧羊犬 + average_dog_weight: 苏格兰梗犬\nPAUSE（暂停）'

In [17]:
next_prompt = "观察: {}".format(average_dog_weight("边境牧羊犬"))
print(next_prompt)

观察: 边境牧羊犬的平均体重是37磅


In [18]:
abot(next_prompt)

'答案：计算两只狗的总重量需要知道每只狗的具体体重。但是，如果我们使用边境牧羊犬和苏格兰梗犬的平均体重进行估算：\n\n- 假设边境牧羊犬的平均体重为37磅。\n- 苏格兰梗犬的平均体重通常在20到35磅之间。\n\n我们可以假设苏格兰梗犬的平均体重为28磅（取中间值）作为估算。\n\n那么，两只狗的总重量大约是：\n\n计算: 2 * (37 + 28) = 146 磅\n\n答案：边境牧羊犬和苏格兰梗犬的总重量约为146磅。请注意，这个结果基于平均体重进行估算，并且实际重量可能会有所不同。'

In [19]:
next_prompt = "观察: {}".format(average_dog_weight("苏格兰梗犬"))
print(next_prompt)

观察: 苏格兰梗犬的平均体重是20磅。


In [20]:
abot(next_prompt)

'思考：现在我有了两只狗的具体平均体重。\n行动：calculate: 37 + 20\nPAUSE（暂停）'

In [21]:
next_prompt = "观察: {}".format(eval("37 + 20"))
print(next_prompt)

观察: 57


In [22]:
abot(next_prompt)

'答案：边境牧羊犬和苏格兰梗犬的总重量约为57磅。'

### Add loop 

In [23]:
import re

action_re = re.compile('^行动: (\w+): (.*)$')   # python regular expression to selection action

In [24]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("不知行动: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "观察: {}".format(observation)
        else:
            return

In [25]:
question = """我有两只狗，一只边境牧羊犬和一只苏格兰梗犬。它们的总重量是多少？"""
query(question)

思考：我需要计算两只狗的总重量。
行动：calculate: 2 * average_dog_weight(边境牧羊犬) + average_dog_weight(苏格兰梗犬)
PAUSE（暂停）
